In [1]:
import numpy as np
import pandas as pd
from scipy.stats import iqr
from sklearn.neural_network import MLPClassifier

In [39]:
r_x = 3
r_e = 1
r_xe = r_x+r_e
r_z = (10,5)
r_ez = (10,5)
r_y = 3
n = 10000

In [21]:


choose = lambda x,y: int(np.product(np.arange(x,x-y,-1))/np.product(np.arange(1,y+1)))
bind = lambda *x: np.hstack(x) 
relu = lambda x: np.maximum(x,0)
softmax = lambda x: np.divide(np.exp(x),np.exp(x).sum(1).reshape(-1,1))

ones = np.ones((n,1))
w = np.ones((1,2))
for i in range(2,r_xe):
    w = bind(bind(np.ones((i,1)),np.eye(i)).T,\
             bind(np.zeros((w.shape[0],1)),w).T).T
xe = np.random.normal(size=(n,r_xe))
z = np.random.normal(size=(n,choose(r_xe,2)))
for i in range(w.shape[0]):
    if w[i,-r_e:].sum() != 0 and w[i,:r_x].sum() != 0:
        w[i,:] = 0
xe = xe + z@w

In [27]:
def mypred(x):
    last = x.copy()
    ones = np.ones((x.shape[0],1))
    for i in range(len(wmat)):
        w_tmp = wmat[i]
        last = relu(bind(ones,last)@w_tmp)
    return softmax(last)

while True:
    wmat = []
    layers = []
    last = xe
    for i in range(len(r_z)):
        w_tmp = np.random.uniform(size=(last.shape[1]+1,r_z[i]))*3-1
        last = relu(bind(ones,last)@w_tmp)
        wmat += [w_tmp]
        layers += [last]
    w_tmp = np.random.uniform(size=(last.shape[1]+1,r_y))*3-1
    last = relu(bind(ones,last)@w_tmp)
    wmat += [w_tmp]
    layers += [last]
    y = softmax(last).argmax(1)
    (values,counts) = np.unique(y,return_counts=True)
    if values.shape[0] == 3 and counts.min()>0.1*n: break
np.unique(y,return_counts=True)

(array([0, 1, 2]), array([5276, 3594, 1130]))

In [5]:
softmax(mypred(bind(x,np.zeros((n,1)))))

NameError: name 'x' is not defined

In [28]:
x = xe[:,:r_x]
dx = iqr(x,0)/10000
dydx = []
for i in range(r_x):
    x_tmp = x.copy()
    x_tmp[:,i] += dx[i]
    y0 = mypred(bind(x,np.zeros((n,1))))
    y1 = mypred(bind(x_tmp,np.zeros((n,1))))
    y0 /= y0.max(1).reshape(-1,1)
    y1 /= y1.max(1).reshape(-1,1)
    dy = y1-y0
    dydx += [(dy/dx[i]).mean(0)]
dydx = np.array(dydx)
#dydx /= np.sqrt(((dydx)**2).mean())
dydx

array([[ 0.14975793, -0.18673661,  0.0330263 ],
       [-0.05047949, -0.03696358,  0.09885803],
       [-0.06272637,  0.04899884,  0.01982084]])

In [40]:
mlp = MLPClassifier(hidden_layer_sizes=r_ez,max_iter=10000,alpha=1)
model = mlp.fit(x,y)
accur = (y==model.predict(x)).astype(int).sum()/n
accur

0.7743

In [41]:
x = xe[:,:r_x]
dx = iqr(x,0)/10000
edydx = []
for i in range(r_x):
    x_tmp = x.copy()
    x_tmp[:,i] += dx[i]
    y0 = model.predict_proba(x)
    y1 = model.predict_proba(x_tmp)
    #y0 /= y0.sum(1).reshape(-1,1)
    #y1 /= y1.sum(1).reshape(-1,1)
    dy = y1-y0
    edydx += [(dy/dx[i]).mean(0)]
edydx = np.array(edydx)
edydx

array([[ 0.13550244, -0.16126367,  0.02576122],
       [-0.03379881, -0.02654145,  0.06034026],
       [-0.05664603,  0.04980463,  0.00684139]])

In [42]:
((edydx-dydx)**2/(dydx)**2).mean()

0.09500467482362275

In [ ]:
sedydxs = []
nsim = 1000
for i in range(nsim):
    print(i)
    samp = np.random.randint(n,size=(n,))
    submodel = mlp.fit(x[samp,:],y[samp])
    sedydx = []
    for i in range(r_x):
        x_tmp = x[samp,:].copy()
        x_tmp[:,i] += dx[i]
        y0 = submodel.predict_proba(x[samp,:])
        y1 = submodel.predict_proba(x_tmp)
        y0 /= y0.sum(1).reshape(-1,1)
        y1 /= y1.sum(1).reshape(-1,1)
        dy = y1-y0
        sedydx += [(dy/dx[i]).mean(0)]
    sedydx = np.array(sedydx)
    sedydxs += [sedydx]

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54


/opt/conda/lib/python3.7/site-packages/sklearn/neural_network/multilayer_perceptron.py:568: UserWarning: Training interrupted by user.
  warnings.warn("Training interrupted by user.")


55


/opt/conda/lib/python3.7/site-packages/sklearn/neural_network/multilayer_perceptron.py:568: UserWarning: Training interrupted by user.
  warnings.warn("Training interrupted by user.")
/opt/conda/lib/python3.7/site-packages/sklearn/neural_network/multilayer_perceptron.py:568: UserWarning: Training interrupted by user.
  warnings.warn("Training interrupted by user.")


56
57
58


...

60


/opt/conda/lib/python3.7/site-packages/sklearn/neural_network/multilayer_perceptron.py:568: UserWarning: Training interrupted by user.
  warnings.warn("Training interrupted by user.")
/opt/conda/lib/python3.7/site-packages/sklearn/neural_network/multilayer_perceptron.py:568: UserWarning: Training interrupted by user.
  warnings.warn("Training interrupted by user.")


61
62
63


/opt/conda/lib/python3.7/site-packages/sklearn/neural_network/multilayer_perceptron.py:568: UserWarning: Training interrupted by user.
  warnings.warn("Training interrupted by user.")


64
65


In [10]:
sedydxs = np.array(sedydxs)
lower = np.zeros(dydx.shape)
upper = np.zeros(dydx.shape)
from itertools import product
for i,j in product(range(dydx.shape[0]),range(dydx.shape[1])):
    tmparray = sedydxs[:,i,j].copy()
    tmparray = np.sort(tmparray)
    lower[i,j] = tmparray[int(5*nsim/100)]
    upper[i,j] = tmparray[int(95*nsim/100)]
print(lower)
print(upper)

[[ 0.0818888  -0.10014801  0.00426798]
 [ 0.11868388  0.0234094  -0.15514002]
 [ 0.07072995  0.03011083 -0.11177185]]
[[ 0.09016281 -0.08970921  0.01431783]
 [ 0.12877022  0.03058297 -0.14520126]
 [ 0.07845098  0.03741305 -0.10322625]]


In [11]:
dydx

array([[ 0.06081579, -0.14428851,  0.0172218 ],
       [ 0.14331452,  0.04042013, -0.0973202 ],
       [ 0.07447887,  0.05772818, -0.05119661]])